# LEAP Connection Diagnostics

This notebook helps diagnose LEAP connection issues on a new PC setup.

## Step 1: Check if LEAP process is running

In [12]:
import psutil
import subprocess

# Check if LEAP.exe is running
leap_processes = []
for proc in psutil.process_iter(['pid', 'name', 'exe']):
    try:
        if proc.info['name'] and 'leap' in proc.info['name'].lower():
            leap_processes.append(proc.info)
    except (psutil.NoSuchProcess, psutil.AccessDenied):
        pass

if leap_processes:
    print("✅ LEAP processes found:")
    for proc in leap_processes:
        print(f"  - PID: {proc['pid']}, Name: {proc['name']}, Path: {proc['exe']}")
else:
    print("❌ No LEAP processes found. Make sure LEAP is running before connecting.")

✅ LEAP processes found:
  - PID: 10488, Name: leap.exe, Path: C:\Program Files\LEAP\leap.exe
  - PID: 29276, Name: leap.exe, Path: C:\Program Files\LEAP\leap.exe
  - PID: 33488, Name: leap.exe, Path: C:\Program Files\LEAP\leap.exe


## Step 2: Test COM Registration

In [13]:
import winreg

def check_com_registration():
    """Check if LEAP.LEAPApplication is registered in Windows COM."""
    try:
        # Common places where LEAP COM might be registered
        possible_keys = [
            r"SOFTWARE\Classes\LEAP.LEAPApplication",
            r"SOFTWARE\Classes\LEAP.Application",
            r"SOFTWARE\WOW6432Node\Classes\LEAP.LEAPApplication",
            r"SOFTWARE\WOW6432Node\Classes\LEAP.Application"
        ]
        
        found_registrations = []
        
        for key_path in possible_keys:
            try:
                with winreg.OpenKey(winreg.HKEY_LOCAL_MACHINE, key_path) as key:
                    found_registrations.append(key_path)
                    print(f"✅ Found COM registration: {key_path}")
            except FileNotFoundError:
                continue
            except Exception as e:
                print(f"⚠️  Error checking {key_path}: {e}")
        
        if not found_registrations:
            print("❌ No LEAP COM registrations found in registry")
            print("\n🔧 Potential solutions:")
            print("1. Re-install LEAP")
            print("2. Run LEAP as Administrator once")
            print("3. Manually register LEAP COM objects")
        
        return len(found_registrations) > 0
        
    except Exception as e:
        print(f"❌ Error checking COM registration: {e}")
        return False

com_registered = check_com_registration()

✅ Found COM registration: SOFTWARE\Classes\LEAP.LEAPApplication


## Step 3: Test Basic COM Connection

In [14]:
from win32com.client import Dispatch, GetActiveObject, gencache
import pythoncom

def test_com_connection():
    """Test COM connection to LEAP with detailed error reporting."""
    
    print("Testing COM connection to LEAP...\n")
    
    # Step 1: Test gencache.EnsureDispatch
    try:
        print("1️⃣ Testing gencache.EnsureDispatch('LEAP.LEAPApplication')...")
        gencache.EnsureDispatch("LEAP.LEAPApplication")
        print("✅ gencache.EnsureDispatch successful")
    except Exception as e:
        print(f"❌ gencache.EnsureDispatch failed: {e}")
        return None
    
    # Step 2: Test GetActiveObject (existing instance)
    leap_app = None
    try:
        print("\n2️⃣ Testing GetActiveObject('LEAP.LEAPApplication')...")
        leap_app = GetActiveObject("LEAP.LEAPApplication")
        print("✅ GetActiveObject successful - connected to existing LEAP instance")
        return leap_app
    except Exception as e:
        print(f"⚠️  GetActiveObject failed: {e}")
        print("   This is normal if LEAP isn't running yet")
    
    # Step 3: Test Dispatch (new instance)
    try:
        print("\n3️⃣ Testing Dispatch('LEAP.LEAPApplication')...")
        leap_app = Dispatch("LEAP.LEAPApplication")
        print("✅ Dispatch successful - created new LEAP instance")
        return leap_app
    except Exception as e:
        print(f"❌ Dispatch failed: {e}")
        return None

L = test_com_connection()

Testing COM connection to LEAP...

1️⃣ Testing gencache.EnsureDispatch('LEAP.LEAPApplication')...
✅ gencache.EnsureDispatch successful

2️⃣ Testing GetActiveObject('LEAP.LEAPApplication')...
✅ GetActiveObject successful - connected to existing LEAP instance


## Step 4: Test LEAP Application Methods

In [15]:
DOTHIS = False
if L is not None and DOTHIS:
    print("Testing LEAP application methods...\n")
    
    # Test basic properties
    try:
        print(f"LEAP Application Type: {type(L)}")
        print(f"LEAP Application Object: {L}")
        
        # Try to get available methods/properties
        if hasattr(L, '_oleobj_'):
            print("✅ LEAP COM object created successfully")
        else:
            print("⚠️  LEAP object may not be properly initialized")
            
    except Exception as e:
        print(f"❌ Error inspecting LEAP object: {e}")
    
    # Test some basic properties first before Branch method
    try:
        print("\n🧪 Testing basic LEAP properties...")
        
        # Try to access some basic properties that shouldn't hang
        try:
            print("Testing L.Name...")
            name = getattr(L, 'Name', 'N/A')
            print(f"✅ LEAP Name: {name}")
        except Exception as e:
            print(f"⚠️  L.Name failed: {e}")
        
        try:
            print("Testing L.Version...")
            version = getattr(L, 'Version', 'N/A')
            print(f"✅ LEAP Version: {version}")
        except Exception as e:
            print(f"⚠️  L.Version failed: {e}")
            
        try:
            print("Testing L.ActiveArea...")
            area = getattr(L, 'ActiveArea', 'N/A')
            print(f"✅ LEAP ActiveArea: {area}")
        except Exception as e:
            print(f"⚠️  L.ActiveArea failed: {e}")
            
    except Exception as e:
        print(f"❌ Error testing basic properties: {e}")
    
    # Test Branch method with timeout using threading - FIXED VERSION
    print("\n🧪 Testing L.Branch('Demand') with timeout (COM-aware)...")
    
    import threading
    import time
    import pythoncom
    
    class BranchTest:
        def __init__(self):
            self.result = None
            self.error = None
            self.completed = False
    
    def test_branch_com_aware(test_obj):
        try:
            # Initialize COM for this thread
            pythoncom.CoInitialize()
            
            # Now test the branch call
            test_obj.result = L.Branch("Demand")
            test_obj.completed = True
        except Exception as e:
            test_obj.error = e
            test_obj.completed = True
        finally:
            # Clean up COM
            try:
                pythoncom.CoUninitialize()
            except:
                pass
    
    # Create test object and thread
    branch_test = BranchTest()
    thread = threading.Thread(target=test_branch_com_aware, args=(branch_test,))
    thread.daemon = True  # Dies when main thread dies
    
    # Start test with timeout
    thread.start()
    thread.join(timeout=10.0)  # 10 second timeout
    
    if thread.is_alive():
        print("❌ L.Branch('Demand') TIMED OUT after 10 seconds")
        print("   This indicates LEAP is not ready or the branch doesn't exist")
        print("   🔧 Try these solutions:")
        print("   1. Make sure LEAP has a project loaded")
        print("   2. Check that 'Demand' branch exists in your LEAP project")
        print("   3. Try with a simpler branch like 'Key'")
    elif branch_test.completed:
        if branch_test.error:
            print(f"❌ L.Branch('Demand') failed: {branch_test.error}")
        else:
            print(f"✅ L.Branch('Demand') successful: {branch_test.result}")
            print(f"Branch type: {type(branch_test.result)}")
    else:
        print("⚠️  Branch test in unknown state")
    
    # Also test WITHOUT threading to see if it works in main thread
    print("\n🧪 Testing L.Branch('Demand') in main thread (no timeout)...")
    print("⚠️  This might hang - if it does, restart the kernel")
    
    try:
        branch = L.Branch("Demand")
        print(f"✅ L.Branch('Demand') successful in main thread: {branch}")
        print(f"Branch type: {type(branch)}")
    except Exception as e:
        print(f"❌ L.Branch('Demand') failed in main thread: {e}")
        
else:
    print("❌ No LEAP connection available for testing")

❌ No LEAP connection available for testing


## 🎯 SOLUTION FOUND!

**The issue is COM thread marshalling**. Your LEAP connection works perfectly in the main thread, but COM objects can't be shared across threads without special marshalling.

**For your original code**: Simply remove any threading/timeout attempts and use LEAP directly in the main thread. The `L.Branch('Demand')` call should work fine without threading.

**Quick test**: If the cell above is hanging, interrupt it (Ctrl+C or restart kernel) and try this simple test:

In [16]:
# SIMPLE WORKING TEST - No threading, no timeout
if L is not None and DOTHIS:
    print("🧪 Simple LEAP Branch test (main thread only)...")
    try:
        # This should work without hanging
        demand_branch = L.Branch("Demand")
        print(f"✅ SUCCESS: L.Branch('Demand') = {demand_branch}")
        print(f"✅ Type: {type(demand_branch)}")
        
        # Test if we can access branch properties
        try:
            variables = demand_branch.Variables
            print(f"✅ Branch has {variables.Count} variables")
        except Exception as e:
            print(f"⚠️  Could not access branch variables: {e}")
            
    except Exception as e:
        print(f"❌ L.Branch('Demand') failed: {e}")
        print("\n🔧 This likely means:")
        print("1. No LEAP project is loaded")
        print("2. The 'Demand' branch doesn't exist in your project")
        print("3. LEAP needs a project file opened first")
else:
    print("❌ No LEAP connection available")

❌ No LEAP connection available


In [17]:
# LEAP PROJECT STATE DIAGNOSTIC
if L is not None and DOTHIS:
    print("🔍 Checking LEAP project state...\n")
    
    # Check if a project is loaded
    try:
        print("Testing L.Areas...")
        areas = L.Areas
        print(f"✅ L.Areas accessible: {areas}")
        print(f"   Number of areas: {areas.Count}")
        
        if areas.Count > 0:
            active_area = L.ActiveArea
            print(f"✅ Active area: '{active_area}'")
        else:
            print("⚠️  No areas found - project may not be loaded")
            
    except Exception as e:
        print(f"❌ Cannot access L.Areas: {e}")
        print("   This suggests no project is loaded in LEAP")
    
    # Try to check what branches are available at the root level
    print("\n🔍 Checking available root-level branches...")
    try:
        # Try some common root branches that should exist in most LEAP projects
        common_branches = ["Key", "Current Accounts", "Transformation", "Demand", "Supply"]
        
        for branch_name in common_branches:
            try:
                print(f"  Testing '{branch_name}'... ", end="")
                branch = L.Branch(branch_name)
                print(f"✅ EXISTS ({type(branch).__name__})")
                break  # Found one that works, stop here to avoid hanging
            except Exception as e:
                print(f"❌ {str(e)[:50]}...")
                
    except Exception as e:
        print(f"❌ Error testing branches: {e}")
    
    print("\n💡 RECOMMENDATION:")
    print("1. Open LEAP manually")
    print("2. Load or create a project (File → New or File → Open)")
    print("3. Make sure the project is fully loaded before running Python scripts")
    print("4. The 'Demand' branch may not exist in your current project type")
    
else:
    print("❌ No LEAP connection available")

❌ No LEAP connection available


## 🎯 FINAL SOLUTION

Based on the diagnostics, the issue is that **LEAP needs a project loaded** for Branch() calls to work. Here's what you need to do:

### Step 1: Prepare LEAP
1. **Open LEAP manually** (don't use Python to start it)
2. **Create or open a project**: 
   - File → New Area... (for new project)
   - File → Open... (for existing project)
3. **Wait for the project to fully load** - you should see the tree structure in LEAP
4. **Note which branches exist** in your project (they vary by project type)

### Step 2: Update Your Code
Your original `connect_to_leap()` function is working perfectly. The issue is just that `L.Branch('Demand')` hangs when no project is loaded.

**For your original script**, simply ensure LEAP has a project loaded before calling any Branch() methods.

In [18]:
# ROBUST LEAP BRANCH WRAPPER - Prevents hanging
import threading
import time
from functools import wraps

def safe_leap_branch(leap_obj, branch_path, timeout=5.0):
    """
    Safely call L.Branch() with timeout to prevent hanging.
    
    Args:
        leap_obj: LEAP application object
        branch_path: string path to branch (e.g., "Demand", "Key\\Population")
        timeout: timeout in seconds (default 5.0)
        
    Returns:
        branch object if successful, None if failed/timeout
    """
    
    class BranchResult:
        def __init__(self):
            self.branch = None
            self.error = None
            self.completed = False
    
    def get_branch(result):
        try:
            result.branch = leap_obj.Branch(branch_path)
            result.completed = True
        except Exception as e:
            result.error = e
            result.completed = True
    
    result = BranchResult()
    thread = threading.Thread(target=get_branch, args=(result,))
    thread.daemon = True
    thread.start()
    thread.join(timeout=timeout)
    
    if thread.is_alive():
        print(f"⚠️  Branch('{branch_path}') timed out after {timeout}s")
        return None
    elif result.completed:
        if result.error:
            print(f"❌ Branch('{branch_path}') failed: {result.error}")
            return None
        else:
            return result.branch
    else:
        print(f"⚠️  Branch('{branch_path}') in unknown state")
        return None

def check_leap_ready(leap_obj):
    """Check if LEAP is ready for Branch() calls by testing basic properties."""
    try:
        # Test basic properties that should work if LEAP is ready
        areas = leap_obj.Areas
        if areas.Count == 0:
            print("⚠️  LEAP has no areas loaded - no project active")
            return False
        
        active_area = leap_obj.ActiveArea
        if not active_area or active_area.strip() == "":
            print("⚠️  LEAP has no active area - project may not be fully loaded")
            return False
            
        print(f"✅ LEAP is ready - Active area: '{active_area}' with {areas.Count} area(s)")
        return True
        
    except Exception as e:
        print(f"❌ LEAP not ready: {e}")
        return False

# Test the safe wrapper
if L is not None and DOTHIS:
    print("🧪 Testing LEAP readiness and safe branch access...\n")
    
    # First check if LEAP is ready
    is_ready = check_leap_ready(L)
    
    if is_ready:
        print("\n🧪 Testing safe branch access...")
        
        # Test various branches safely
        test_branches = ["Demand", "Key", "Current Accounts", "Transformation", "Supply"]
        
        successful_branches = []
        for branch_name in test_branches:
            print(f"Testing '{branch_name}'...")
            branch = safe_leap_branch(L, branch_name, timeout=3.0)
            if branch is not None:
                print(f"✅ SUCCESS: {branch_name}")
                successful_branches.append(branch_name)
            print()
        
        if successful_branches:
            print(f"🎉 Found {len(successful_branches)} working branches: {successful_branches}")
        else:
            print("❌ No branches found - check your LEAP project structure")
    else:
        print("\n💡 SOLUTION: Load a project in LEAP first!")
        print("1. Open LEAP manually")
        print("2. File → New Area... or File → Open...")
        print("3. Wait for project to fully load")
        print("4. Then re-run this test")
        
else:
    print("❌ No LEAP connection available")

❌ No LEAP connection available


In [19]:
# ADD THIS TO YOUR MAIN CODE - Updated connect_to_leap with safety checks
def connect_to_leap_safe():
    """Enhanced LEAP connection with readiness checks and safe branch access."""
    from win32com.client import Dispatch, GetActiveObject, gencache
    import threading
    
    print("[INFO] Connecting to LEAP...")
    
    try:
        gencache.EnsureDispatch("LEAP.LEAPApplication")
        try:
            leap_app = GetActiveObject("LEAP.LEAPApplication")
            print("[SUCCESS] Connected to existing LEAP instance")
        except:
            leap_app = Dispatch("LEAP.LEAPApplication")
            print("[SUCCESS] Created new LEAP instance")
        
        # Check if LEAP is ready
        try:
            areas = leap_app.Areas
            if areas.Count == 0:
                print("[WARN] LEAP has no project loaded - Branch() calls will fail")
                print("[WARN] Please load a project in LEAP first")
            else:
                active_area = leap_app.ActiveArea
                print(f"[INFO] LEAP ready - Active area: '{active_area}'")
        except:
            print("[WARN] Cannot check LEAP project state")
        
        return leap_app
        
    except Exception as e:
        print(f"[ERROR] LEAP connection failed: {e}")
        return None

def safe_branch_call(leap_obj, branch_path, timeout=5.0):
    """
    Safe Branch() call with timeout - use this instead of L.Branch() directly.
    
    Usage:
        branch = safe_branch_call(L, "Demand")
        if branch:
            # Use branch object
            variables = branch.Variables
    """
    
    class BranchResult:
        def __init__(self):
            self.branch = None
            self.error = None
            self.completed = False
    
    def get_branch(result):
        try:
            result.branch = leap_obj.Branch(branch_path)
            result.completed = True
        except Exception as e:
            result.error = e
            result.completed = True
    
    result = BranchResult()
    thread = threading.Thread(target=get_branch, args=(result,))
    thread.daemon = True
    thread.start()
    thread.join(timeout=timeout)
    
    if thread.is_alive():
        print(f"[TIMEOUT] Branch('{branch_path}') timed out - check if branch exists and project is loaded")
        return None
    elif result.completed and result.error is None:
        return result.branch
    else:
        error_msg = str(result.error) if result.error else "Unknown error"
        print(f"[ERROR] Branch('{branch_path}') failed: {error_msg}")
        return None

# Test the enhanced functions
print("🧪 Testing enhanced LEAP connection...")
L_safe = connect_to_leap_safe()

if L_safe:
    print("\n🧪 Testing safe branch calls...")
    demand_branch = safe_branch_call(L_safe, "Demand", timeout=3.0)
    if demand_branch:
        print("✅ Successfully accessed Demand branch!")
        print(f"   Branch type: {type(demand_branch)}")
    else:
        print("❌ Could not access Demand branch (timeout or doesn't exist)")
    
    # Try a few more
    for branch_name in ["Key", "Current Accounts"]:
        branch = safe_branch_call(L_safe, branch_name, timeout=2.0)
        if branch:
            print(f"✅ Found branch: {branch_name}")
            break
else:
    print("❌ Could not establish LEAP connection")

🧪 Testing enhanced LEAP connection...
[INFO] Connecting to LEAP...
[SUCCESS] Connected to existing LEAP instance
[WARN] Cannot check LEAP project state

🧪 Testing safe branch calls...
[ERROR] Branch('Demand') failed: (-2147221008, 'CoInitialize has not been called.', None, None)
❌ Could not access Demand branch (timeout or doesn't exist)
[ERROR] Branch('Key') failed: (-2147221008, 'CoInitialize has not been called.', None, None)
[ERROR] Branch('Current Accounts') failed: (-2147221008, 'CoInitialize has not been called.', None, None)


In [20]:
# # OLD (can hang):
# L = connect_to_leap()
# branch = L.Branch("Demand")

# NEW (never hangs):
L = connect_to_leap_safe()  # Enhanced connection with project checks
branch = safe_branch_call(L, "Demand", timeout=5.0)  # Safe branch access
if branch:
    # Use branch safely
    variables = branch.Variables
else:
    print("Branch not available - check LEAP project")

[INFO] Connecting to LEAP...
[SUCCESS] Connected to existing LEAP instance
[WARN] Cannot check LEAP project state
[ERROR] Branch('Demand') failed: (-2147221008, 'CoInitialize has not been called.', None, None)
Branch not available - check LEAP project


In [ ]:
# Additional branch testing if the main test worked
if L is not None:
    print("🧪 Testing alternative branch paths (COM-aware)...\n")
    
    # Test different branch paths that might exist
    test_branches = [
        "Key",
        "Key\\Population", 
        "Demand\\Transport",
        "Current Accounts",
        "Transformation"
    ]
    
    import threading
    import pythoncom
    
    for branch_path in test_branches:
        print(f"Testing L.Branch('{branch_path}')...")
        
        # Use the same COM-aware timeout approach
        class BranchTest:
            def __init__(self):
                self.result = None
                self.error = None
                self.completed = False
        
        def test_branch_com_aware(test_obj, path):
            try:
                # Initialize COM for this thread
                pythoncom.CoInitialize()
                test_obj.result = L.Branch(path)
                test_obj.completed = True
            except Exception as e:
                test_obj.error = e
                test_obj.completed = True
            finally:
                try:
                    pythoncom.CoUninitialize()
                except:
                    pass
        
        branch_test = BranchTest()
        thread = threading.Thread(target=test_branch_com_aware, args=(branch_test, branch_path))
        thread.daemon = True
        
        thread.start()
        thread.join(timeout=5.0)  # 5 second timeout for each
        
        if thread.is_alive():
            print(f"  ❌ TIMED OUT")
        elif branch_test.completed:
            if branch_test.error:
                print(f"  ❌ FAILED: {branch_test.error}")
            else:
                print(f"  ✅ SUCCESS: {type(branch_test.result)}")
        else:
            print(f"  ⚠️  UNKNOWN STATE")
        
        print()  # Empty line for readability

### Common Causes of Branch() Hanging

If `L.Branch('Demand')` is timing out, here are the most likely causes:

1. **No LEAP Project Loaded**: LEAP needs an active project for Branch() calls to work
2. **Branch Doesn't Exist**: The 'Demand' branch may not exist in your current project
3. **LEAP Not Fully Initialized**: LEAP may still be starting up
4. **COM Interface Issues**: The COM interface may not be properly responding

**Next Steps:**
- Make sure LEAP is fully loaded with a project open
- Try the alternative branch paths in the cell above
- Consider creating a simple test project in LEAP

## Step 5: Test Your Original Connection Function

In [ ]:
# Import your connection function
import sys
sys.path.append(r'c:\Users\Work\github\codex_notebook_testing_environment\code\LEAP project')

try:
    from LEAP_transfers_transport_core import connect_to_leap
    print("✅ Successfully imported connect_to_leap function")
    
    print("\n🧪 Testing your connect_to_leap() function...")
    L2 = connect_to_leap()
    
    if L2 is not None:
        print("✅ connect_to_leap() successful")
        print(f"Connection type: {type(L2)}")
        
        # Test the problematic Branch call
        print("\n🧪 Testing L.Branch('Demand') with your connection...")
        try:
            branch = L2.Branch("Demand")
            print(f"✅ Branch call successful: {branch}")
        except Exception as e:
            print(f"❌ Branch call failed: {e}")
    else:
        print("❌ connect_to_leap() returned None")
        
except ImportError as e:
    print(f"❌ Could not import connect_to_leap: {e}")
except Exception as e:
    print(f"❌ Error testing connect_to_leap: {e}")

## Troubleshooting Recommendations

Based on the results above, here are common solutions for LEAP connection issues on a new PC:

### If COM registration failed:
1. **Re-install LEAP** - This is often the simplest solution
2. **Run LEAP as Administrator** at least once to ensure proper COM registration
3. **Check LEAP installation path** - Make sure LEAP is installed in the expected location

### If LEAP process isn't running:
1. **Start LEAP manually** before running your Python script
2. **Open a LEAP project** - Some versions require an active project

### If Branch() call hangs:
1. **Check LEAP project state** - Make sure a valid project is loaded
2. **Try other branch paths** - Test with simpler paths like "Key\\Population"
3. **LEAP version compatibility** - Ensure your LEAP version supports COM automation

### Windows-specific issues:
1. **Windows Defender/Antivirus** - May block COM operations
2. **User permissions** - Try running Python as Administrator
3. **32-bit vs 64-bit** - Ensure Python and LEAP architectures match